# Project Work Hand-on Python for ML: MobileNet e Bi-LSTM per la Violence Detection su dataset "AIRTLab"

Questo notebook estende gli esperimenti presentati in
> P. Sernani, N. Falcionelli, S. Tomassini, P. Contardo and A. F. Dragoni, "Deep Learning for Automatic Violence Detection: Tests on the AIRTLab Dataset," in IEEE Access, vol. 9, pp. 160580-160595, 2021, doi: 10.1109/ACCESS.2021.3131315.

Il paper è open access e disponibile qui: [https://ieeexplore.ieee.org/document/9627980](https://ieeexplore.ieee.org/document/9627980).

In particolare, l'esperimento testa una **Convolutional Neural Network (CNN) 2D pre-addestrata su Imagenet**, più leggera rispetto a quelli presentati nel paper:

- MobileNetV2 ([https://keras.io/api/applications/mobilenet/#mobilenetv2-function](https://keras.io/api/applications/mobilenet/#mobilenetv2-function))

In questo notebook, MobileNetV2 viene combinata con una **Bi-LSTM** e dei **layer fully connected** per la classificazione di sequenze di frame dei video del **dataset "AIRTLab"** in violente e non-violente.

Il dataset è open access e disponibile al seguente link:
> <https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos>

## 1 Operazioni preliminari e Data Pre-Processing (DPP)
Le celle seguenti:
- **clonano il dataset AIRTLab** nella directory /datarepo;
- definiscono le funzioni per il **DPP**, effettuando il **ridimensionamento** dei frame dei video a 224 x 224 (formato usato per l'input di MobileNetV2) e **dividendo i video** in sottosequenze di 16 frame.

In [ ]:
# Scarica il dataset AIRTLab pensato per testare tecniche di Violence Detection
# !mkdir /datarepo
# !git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos.git /datarepo

In [ ]:
# Monto drive Google

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/apple/ml-mobileone.git


Cloning into 'ml-mobileone'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 54
Receiving objects: 100% (56/56), 749.78 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
import torch
import sys

sys.path.append('/content/ml-mobileone')


In [ ]:
from mobileone import mobileone

model = mobileone(variant='s0', inference_mode=True)
checkpoint = torch.load('/content/gdrive/MyDrive/checkpoint.pth.tar')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
import tensorflow as tf
import torch
from mobileone import mobileone

class PyTorchWrapper(tf.keras.layers.Layer):
    def __init__(self, variant='s0', **kwargs):
        super(PyTorchWrapper, self).__init__(**kwargs)
        self.model = mobileone(variant=variant, inference_mode=True)
        self.model.eval()  # Assicurati che il modello sia in modalità valutazione

    def call(self, inputs):
        # Converte i tensori di input in tensori PyTorch
        inputs = torch.tensor(inputs.numpy())
        # Passa gli input attraverso il modello PyTorch
        outputs = self.model(inputs)
        # Converte i tensori di output da PyTorch a TensorFlow
        outputs = tf.convert_to_tensor(outputs.detach().numpy())
        return outputs

    def compute_output_shape(self, input_shape):
        # La forma di output è la stessa della forma di input
        return input_shape

# Esempio di utilizzo del wrapper PyTorch con TimeDistributed
input_shape = (None, 10, 32)  # Ad esempio, input shape (batch_size, time_steps, features)
model = PyTorchWrapper(variant='s0')

In [ ]:
# Sistemazione del Dataset AirtLab
# import os
# import shutil

# def move_files(dataset_path):
#     # Definisci i percorsi delle cartelle "Violence" e "NonViolence"
#     violence_path = os.path.join(dataset_path, "Violence")
#     nonviolence_path = os.path.join(dataset_path, "NonViolence")

#     # Funzione per spostare i file da cam2 alla cartella superiore
#     def move_from_cam2(folder_path):
#         cam2_path = os.path.join(folder_path, "cam2")
#         if os.path.exists(cam2_path) and os.path.isdir(cam2_path):
#             for filename in os.listdir(cam2_path):
#                 source_path = os.path.join(cam2_path, filename)
#                 if os.path.isfile(source_path):
#                     dest_path = os.path.join(folder_path, filename)
#                     # Controlla se il file esiste già nella cartella superiore
#                     if os.path.exists(dest_path):
#                         # Aggiungi il suffisso "-1" al nome del file
#                         base, ext = os.path.splitext(filename)
#                         dest_path = os.path.join(folder_path, base + "-1" + ext)
#                     # Sposta il file
#                     shutil.move(source_path, dest_path)
#             # Rimuove la cartella cam2 se è vuota
#             if not os.listdir(cam2_path):
#                 os.rmdir(cam2_path)

#     # Sposta i file da cam2 per entrambe le cartelle
#     move_from_cam2(violence_path)
#     move_from_cam2(nonviolence_path)


# # Percorso alla cartella principale del dataset
# dataset_path = '/content/gdrive/My Drive/Dataset/AirtLab-Dataset'
# move_files(dataset_path)

In [ ]:
# # Unione dei due Dataset disponibili, Kaggle e AirtLab

# def merge_datasets(src1, src2, dst):
#     # Creazione delle cartelle di destinazione "Violence" e "NonViolence"
#     os.makedirs(os.path.join(dst, "Violence"), exist_ok=True)
#     os.makedirs(os.path.join(dst, "NonViolence"), exist_ok=True)

#     # Funzione per unire i contenuti delle cartelle sorgente nella cartella di destinazione
#     def merge_category(src_path, dst_path):
#         for filename in os.listdir(src_path):
#             src_file_path = os.path.join(src_path, filename)
#             if os.path.isfile(src_file_path):
#                 dest_file_path = os.path.join(dst_path, filename)
#                 # Controlla se il file esiste già nella cartella di destinazione
#                 if os.path.exists(dest_file_path):
#                     # Aggiungi il suffisso "-1" al nome del file
#                     base, ext = os.path.splitext(filename)
#                     dest_file_path = os.path.join(dst_path, base + "-1" + ext)
#                 # Copia il file nella cartella di destinazione
#                 shutil.copy2(src_file_path, dest_file_path)

#     # Unisci i contenuti delle cartelle "Violence"
#     merge_category(os.path.join(src1, "Violence"), os.path.join(dst, "Violence"))
#     merge_category(os.path.join(src2, "Violence"), os.path.join(dst, "Violence"))

#     # Unisci i contenuti delle cartelle "NonViolence"
#     merge_category(os.path.join(src1, "NonViolence"), os.path.join(dst, "NonViolence"))
#     merge_category(os.path.join(src2, "NonViolence"), os.path.join(dst, "NonViolence"))

# # Percorsi alle cartelle sorgenti e alla cartella di destinazione
# src1 = '/content/gdrive/My Drive/Dataset/KaggleViolenceNonViolence'
# src2 = '/content/gdrive/My Drive/Dataset/AirtLab-Dataset'
# dst = '/content/gdrive/My Drive/Dataset/Dataset-Edge'

# # Unisci i dataset
# merge_datasets(src1, src2, dst)

In [ ]:
# Funzioni per DPP e utilità (chunk count, video preprocessing, feature computation, ...)
import os
import cv2
import numpy as np

def count_chunks(videoBasePath):
    """Counts the 16 frames lenght chunks available in a dataset organized in violent and non-violent,
    cam1 and cam2 folders, placed at videoBasePath.

    Parameters
    ----------
    videoBasePath : str
                    Base path of the dataset

    Returns
    -------
    cnt : int
          number of 16 frames lenght chunks in the dataset
    """

    folders = ['Violence', 'NonViolence']
    cams = ['cam1', 'cam2']
    cnt = 0

    for folder in folders:
        for camName in cams:
            path = os.path.join(videoBasePath, folder, camName)

            videofiles = os.listdir(path)
            for videofile in videofiles:
                filePath = os.path.join(path, videofile)
                video = cv2.VideoCapture(filePath)
                numframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = int(video.get(cv2.CAP_PROP_FPS))
                chunks = numframes//16
                cnt += chunks


    return cnt

def preprocessVideos(videoBasePath, mainDir, featureBasePath, verbose=True):
    """Preproccess all the videos.

    It extracts samples from the videos organised in violent and non-violent, cam1 and cam2 folders.
    The samples and the labels are store on two memmap numpy arrays, called samples.mmap and labels.mmap, at "featureBasePath".
    The numpy array with samples has shape (Chunk #, 16, 224, 224, 3), the labels array has shape (Chunk # 16, 224, 224, 3).
    For the AIRTLab dataset the number of chunks is 3537.

    Parameters
    ----------
    videoBasePath : str
                    Pathname to the base of the video repository, which contains two directories,
                    violent and non-violent, which are divided into cam1 and cam2.
    mainDir: str
             Pathaname to store the files with sample filenames and labels.
    featureBasePath : str
                      it is the pathname of a base where the numpy arrays have to be saved.
    verbose : bool
              if True print debug logs (default True)

    """

    folders = ['Violence', 'NonViolence']
    cams = ['cam1', 'cam2']
    total_chunks = count_chunks(videoBasePath)
    fileNames = []
    npLabels = np.zeros(total_chunks)
    cnt = 0

    for folder in folders:
        for camName in cams:
            path = os.path.join(videoBasePath, folder, camName)

            videofiles = os.listdir(path)
            for videofile in videofiles:
                filePath = os.path.join(path, videofile)
                video = cv2.VideoCapture(filePath)
                numframes = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = int(video.get(cv2.CAP_PROP_FPS))
                chunks = numframes//16
                if verbose:
                    print(filePath)
                    print("*** [Video Info] Number of frames: {} - fps: {} - chunks: {}".format(numframes, fps, chunks))
                vid = []
                videoFrames = []
                while True:
                    ret, img = video.read()
                    if not ret:
                        break
                    videoFrames.append(cv2.resize(img, (224, 224)))
                vid = np.array(videoFrames, dtype=np.float32)
                filename = os.path.splitext(videofile)[0]
                chunk_cnt = 0
                for i in range(chunks):
                    X = vid[i*16:i*16+16]
                    chunk_cnt += 1
                    filename = folder + '_' + camName + '_' + videofile + '_chunk_' + str(chunk_cnt) + '.npy'
                    fileNames.append(filename)
                    savepath = os.path.join(featureBasePath, filename)
                    np.save(savepath, np.array(X, dtype=np.float32))
                    if folder == 'Violence':
                        npLabels[cnt] = np.int8(1)
                    else:
                        npLabels[cnt] = np.int8(0)
                    cnt += 1

    fileNamesNp = os.path.join(mainDir, 'filenames.npy')
    np.save(fileNamesNp, fileNames)

    labelsNp = os.path.join(mainDir, 'labels.npy')
    np.save(labelsNp, npLabels)

    if verbose:
        print("** Labels **")
        print(npLabels.shape)
        print('\n****\n')
        print("** Samples **")
        print(len(fileNames))
        print('\n****\n')

    del fileNames
    del npLabels

## 2 Esecuzione Data Pre-Processing (DPP)
Le celle seguenti:
- creano le cartelle dove memorizzare i vettori di feature estratte dalle sottosequenze;
- eseguono il DPP su tutti i video del dataset, trasformandoli in sottosequenze di **16 frame** a **224 x 224**.

In [ ]:
# Cartelle per memorizzare i campioni (le sottosequenze di 16 frame) da usare
# per training, validation e test.
#!rm -rf airtlabDataset
#!mkdir airtlabDataset
#!mkdir airtlabDataset/features
#!mkdir airtlabDataset/results


paths = ["/airtlabDataset", "/airtlabDataset/features", "/airtlabDataset/results"]
for path in paths:
  if not os.path.isdir(path):
    os.mkdir(path)

In [ ]:
preprocessVideos('/content/gdrive/My Drive/Dataset/AirtLab-Dataset', '/airtlabDataset', '/airtlabDataset/features', True)

## 3 Esperimento

The following cells
- define some utilities functions to build the end-to-end models composed of 2D CNNs and Bi-LSTM and to run the experiments on such models; the experiments are tests repeated **5 times** with the **stratified shuffle split** cross-validation scheme. In each split 80% of data are used for training, and 20% of data are used for testing. 12,5% of the training data (i.e. 10% of the entire dataset) is used for validation. In other words, in each test **70%** of data are actually for **training**, **10%** for **validation**, and **20%** for **testing**.
- run the experiments model by model.

Each tested model is composed according to the following table. Note that the weights of the **2D CNNs are the ImageNet weights**, whereas **the other layers are trained on the AIRTLab dataset**.

| Layer Type                                     | Output Shape         | Parameter # |
|:-----------------------------------------------|:---------------------|------------:|
| Time Distributed 2D CNN                        | -                    |      -            |
| Time Distributed Flatten                       | -                   |           0 |
| Bi-LSTM, *128 hidden units*                    | (None, 256)          |           - |
| Dropout, *0.5*                                 | (None, 256)          |           0 |
| Dense, *128 units*, *ReLU activation*          | (None, 128)          |       32896 |
| Dropout, *0.5*                                 | (None, 128)          |           0 |
| Dense,  *1 unit*, *Sigmoid activation*         | (None, 1)            |         129 |

In [ ]:
# definitions of two end-to-end models + definitions of experiments
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import matplotlib.pylab as plt
import os
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, ConvLSTM2D, TimeDistributed, Bidirectional, LSTM
#from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as mobilenet_v2_preprocess_input
from keras.utils import Sequence
import torch
import torch.nn as nn

class DataGen(Sequence):
    """ A sequence of data for training/test/validation, loaded from memory
    batch by batch. Extends the tensorflow.keras.utils.Sequence: https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

    Attributes
    ----------
    base_path : str
                path to the folder including the samples.
    filenames : list<str>
                list of sample filenames.
    labels : list<str>
             list of sample labels.
    batch_size : int
                 batch size to load samples

    """

    def __init__(self, base_path, filenames, labels, batch_size, Preprocess_input):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.Preprocess_input = Preprocess_input

    def __len__(self):
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        return np.array([self.Preprocess_input(np.load(os.path.join(self.base_path, file_name))) for file_name in batch_x]), np.array(batch_y)

def GetPretrainedModel(ModelConstructor, input_shape=(224, 224, 3), print_summary=True):
    """ Builds the VGG16 2D CNN with the Imagenet weights, freezing all layers except layers_to_finetune

    Parameters
    ----------
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    print_summary : bool
                    If True prints the model summary.

    Returns
    -------
    model : Sequential
          The instantiated model.
    """

    model = ModelConstructor # (include_top=False, input_shape=input_shape)

    for param in model.parameters():
      param.requires_grad = False

    return model

def getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    #model.add(LSTM(units=128, return_sequences=False))

    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def getConvLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

def runEndToEndExperiment(getLSTMModel, getConvModel, ModelConstructor, pretrained_input_shape, Preprocess_input, batchSize, datasetBasePath, npyBasePath, featuresPath, samplesMMapName, lablesMMapName, endToEndModelName, rState, savePath):
    """Runs the tests with end to end models.

    Parameters
    ----------
    getLSTMModel : Callable[[Callable[Callable[[bool], [str], [tuple], Sequential],
                   [tuple], [bool], Sequential]], [Callable[[bool], [str], [tuple],
                   Sequential]], [tuple], [bool], Sequential]
                   Function that instantiates the model to be tested. The parameters
                   are a function that returns the Convolutional model to be tested
                   in a time distributed fashion, and a boolean for verbose output
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    batchSize : int
                Batch size to be used for training and testing
    datasetBasePath : str
                      Pathname to the base of the feature files repository,
                      which contains two directories, violent and non-violent,
                      which are divided into cam1 and cam2.
    npyBasePath : str
                  Pathname where the files with sample filenames and labels are
                  stored.
    featuresPath : str
                  Folder containing the actual files with the samples.
    samplesMMapName : str
                      Name of the file storing the list with sample filenames.
    lablesMMapName : str
                     Name of the file storing the list of sample labels.
    endToEndModelName : str
                        Model name to be used in the AUC-ROC plot.
    rState : int, RandomState instance or None
             Controls the randomness of the training and testing indices produced.
             Pass an int for reproducible output across multiple function calls.
    savePath : str
               Path to the directory where the model and weights will be saved.
    """
    chunk_number = count_chunks(datasetBasePath)
    X = np.load(os.path.join(npyBasePath, samplesMMapName))
    y = np.load(os.path.join(npyBasePath, lablesMMapName))

    nsplits = 5
    cv = StratifiedShuffleSplit(n_splits=nsplits, train_size=0.8, random_state=rState)

    tprs = []
    aucs = []
    scores = []
    sens = np.zeros(shape=(nsplits))
    specs = np.zeros(shape=(nsplits))
    f1Scores = np.zeros(shape=(nsplits))
    mean_fpr = np.linspace(0, 1, 100)
    plt.figure(num=1, figsize=(10, 10))
    i = 1

    for train, test in cv.split(X, y):

        X_train, X_val, y_train, y_val = train_test_split(X[train][:], y[train], test_size=0.125, random_state=rState)

        filepath = os.path.join(npyBasePath, featuresPath)

        training_batch_generator = DataGen(filepath, X_train, y_train, batchSize, Preprocess_input)
        validation_batch_generator = DataGen(filepath, X_val, y_val, batchSize, Preprocess_input)
        test_batch_generator = DataGen(filepath, X[test][:], y[test], batchSize, Preprocess_input)

        model = getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape, i == 1)

        # Define the ModelCheckpoint callback to save the best weights
        checkpoint_path = os.path.join(savePath, f"best_model_fold_{i}_epoch_{{epoch:02d}}.h5")
        mc = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True, verbose=1)

        es = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1, restore_best_weights=True)

        model.fit(x=training_batch_generator, validation_data=validation_batch_generator, epochs=50, verbose=1, callbacks=[es, mc])

        del X_train
        del X_val

        print("Computing scores...")
        evaluation = model.evaluate(x=test_batch_generator)
        scores.append(evaluation)
        print("Computing probs...")
        probas = model.predict(x=test_batch_generator, verbose=1).ravel()

        fpr, tpr, thresholds = roc_curve(y[test], probas)
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC split %d (AUC = %0.4f)' % (i, roc_auc))

        y_pred = np.round(probas)
        report = classification_report(y[test], y_pred, target_names=['non-violent', 'violent'], output_dict=True)
        sens[i - 1] = report['violent']['recall']
        specs[i - 1] = report['non-violent']['recall']
        f1Scores[i - 1] = report['violent']['f1-score']

        print('confusion matrix split ' + str(i))
        print(confusion_matrix(y[test], y_pred))
        print(classification_report(y[test], y_pred, target_names=['non-violent', 'violent']))
        print('Loss: ' + str(evaluation[0]))
        print('Accuracy: ' + str(evaluation[1]))
        print('\n')

        # Save the final model of the current fold
        final_model_path = os.path.join(savePath, f'final_model_fold_{i}.h5')
        model.save(final_model_path)

        i += 1

        del report
        del model

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc), lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.title('Cross-Validation ROC of ' + endToEndModelName + ' model', fontsize=18)
    plt.legend(loc="lower right", prop={'size': 15})

    np_scores = np.array(scores)
    losses = np_scores[:, 0:1]
    accuracies = np_scores[:, 1:2]
    print('Losses')
    print(losses)
    print('Accuracies')
    print(accuracies)
    print('Sensitivities')
    print(sens)
    print('Specificities')
    print(specs)
    print('F1-scores')
    print(f1Scores)
    print("Avg loss: {0} +/- {1}".format(np.mean(losses), np.std(losses)))
    print("Avg accuracy: {0} +/- {1}".format(np.mean(accuracies), np.std(accuracies)))
    print("Avg sensitivity: {0} +/- {1}".format(np.mean(sens), np.std(sens)))
    print("Avg specificity: {0} +/- {1}".format(np.mean(specs), np.std(specs)))
    print("Avg f1-score: {0} +/- {1}".format(np.mean(f1Scores), np.std(f1Scores)))

    plt.savefig(endToEndModelName.replace('+', '') + '.pdf')
    plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed
from mobileone_tf import MobileOne


In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_v1_preprocess_input

runEndToEndExperiment(
    getLSTMModel,
    GetPretrainedModel,
    model,
    (224, 224, 3),
    mobilenet_v1_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + BiLSTM',
    42,
    '/content/gdrive/ My Drive/MobileOneS0_BiLSTM'
)


In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_v1_preprocess_input


runEndToEndExperiment(
    getConvLSTMModel,
    GetPretrainedModel,
    model,
    (224, 224, 3),
    mobilenet_v1_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + ConvLSTM',
    42,
    '/content/gdrive/ My Drive/MobileOneS0_ConvLSTM'
)


In [ ]:
from keras import __version__
from keras import backend as K
import sklearn

print('Using Keras version:', __version__, 'backend:', K.backend())

if K.backend() == "tensorflow":
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name == '':
        device_name = "None"
    print('Using TensorFlow version:', tf.__version__, ', GPU:', device_name)

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
!nvidia-smi